# Algoritmo KNN - Modelo Multiclasse

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.ensemble import RandomForestClassifier

In [ ]:
df = pd.read_csv('/content/winequality.csv')
df = df.drop(columns='Unnamed: 0')
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,wine_is_red
0,5.9,0.61,0.08,2.1,0.071,16.0,24.0,0.99376,3.56,0.77,11.1,6,1
1,8.2,1.00,0.09,2.3,0.065,7.0,37.0,0.99685,3.32,0.55,9.0,6,1
2,6.4,0.24,0.49,5.8,0.053,25.0,120.0,0.99420,3.01,0.98,10.5,6,0
3,6.6,0.23,0.26,1.3,0.045,16.0,128.0,0.99340,3.36,0.60,10.0,6,0
4,7.2,0.24,0.29,3.0,0.036,17.0,117.0,0.99411,3.36,0.68,10.1,6,0


In [ ]:
df.quality.value_counts() # tirar 0 e 1: testar tirando e deixando

6    2836
5    2138
7    1079
4     216
8     193
3      30
9       5
Name: quality, dtype: int64

In [ ]:
# igualizando o número de observações de cada classe

df_9 = df[df.quality==9]
df_3 = df[df.quality==3]
df_8 = df[df.quality==8]
df_4 = df[df.quality==4]
df_7 = df[df.quality==7]
df_5 = df[df.quality==5]
df_6 = df[df.quality==6]

In [ ]:
# FORMA RETIRANDO AS CATEGORIAS 9 E 3, COM POUCAS OBSERVACOES

df_8 = resample(df_8, replace = False, n_samples = 193)
df_4 = resample(df_4, replace = False, n_samples = 193)
df_7 = resample(df_7, replace = False, n_samples = 193)
df_5 = resample(df_5, replace = False, n_samples = 193)
df_6 = resample(df_6, replace = False, n_samples = 193)

df1 = pd.concat([df_8,df_4,df_7,df_5,df_6])

df1.quality.value_counts()

8    193
7    193
6    193
5    193
4    193
Name: quality, dtype: int64

In [ ]:
df_8 = resample(df_8, replace = False, n_samples = 100)
df_4 = resample(df_4, replace = False, n_samples = 100)
df_7 = resample(df_7, replace = False, n_samples = 100)
df_5 = resample(df_5, replace = False, n_samples = 100)
df_6 = resample(df_6, replace = False, n_samples = 100)

df = pd.concat([df_9,df_3,df_8,df_4,df_7,df_5,df_6])

df.quality.value_counts()

8    100
7    100
6    100
5    100
4    100
3     30
9      5
Name: quality, dtype: int64

In [ ]:
# normalizando:

for i in df.columns:
  df[i] = (df[i] - df[i].min()) / (df[i].max() - df[i].min())

for i in df1.columns:
  df1[i] = (df1[i] - df1[i].min()) / (df1[i].max() - df1[i].min())

df1.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,wine_is_red
5708,0.267241,0.142857,0.36,0.320,0.025126,0.206642,0.316993,0.316479,0.480315,0.125714,0.654867,1.0,0.0
4692,0.275862,0.371429,0.29,0.705,0.018425,0.140221,0.441176,0.502497,0.299213,0.131429,0.637168,1.0,0.0
1949,0.284483,0.285714,0.31,0.065,0.072027,0.088561,0.071895,0.473783,0.385827,0.291429,0.513274,1.0,1.0
3870,0.318966,0.114286,0.30,0.675,0.072027,0.369004,0.671569,0.740949,0.314961,0.120000,0.088496,1.0,0.0
6126,0.250000,0.133333,0.31,0.310,0.040201,0.221402,0.372549,0.289014,0.220472,0.080000,0.619469,1.0,0.0


In [ ]:
# Separando nosso df em treino e teste:

x = np.array(df.drop(columns='quality'))
y = np.array(df.quality)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

x_train = np.array(x_train) 
x_test = np.array(x_test) 
y_train = np.array(y_train) 
y_test = np.array(y_test)

# BANCO SEM 9 E 3

x1 = df1.drop(columns='quality')
y1 = df1.quality

x1_train, x1_test, y1_train, y1_test = train_test_split(x1, y1, test_size=0.25)

x1_train = np.array(x1_train) 
x1_test = np.array(x1_test) 
y1_train = np.array(y1_train) 
y1_test = np.array(y1_test)

In [ ]:
# distancia = ((a1 - a2)^2 + (b1-b2)^2 + ... + (m1 - m2)^2 )^0.5, sendo a1,...,m1 = treino e a2,...,m2 = teste, m é a 13ª letra do alfabeto, 
# usada porque o x_train possui 13 variáveis (ou dimensões)

d = [np.sqrt(np.sum(np.square((i - x_train)), axis=1)) for i in x_test]

kk = len(d)

# SEM 9 E 3

d1 = [np.sqrt(np.sum(np.square((i - x1_train)), axis=1)) for i in x1_test]

kk1 = len(d1)

print(kk,kk1)

107 242


In [ ]:
resposta = np.array([]) #array das classificações com maior frequência
k = 7 #k vizinhos mais próximos do ponto

for i in range(kk):
  indices = d[i].argsort()[:k] #índice de cada ponto mais próximo 
  neighbors = [] #k vizinhos mais próximos do ponto são guardados aqui
  for j in range(k):
    neighbors.append(y_train[indices[j]]) #classificacao dos pontos
  resposta = np.append(resposta,max(set(neighbors), key=neighbors.count)) #classificação mais frequente

resposta1 = np.array([]) #array das classificações com maior frequência
k1 = 7 #k vizinhos mais próximos do ponto

for i in range(kk1):
  indices1 = d1[i].argsort()[:k] #índice de cada ponto mais próximo 
  neighbors1 = [] #k vizinhos mais próximos do ponto são guardados aqui
  for j in range(k1):
    neighbors1.append(y1_train[indices1[j]]) #classificacao dos pontos
  resposta1 = np.append(resposta1,max(set(neighbors1), key=neighbors1.count)) #classificação mais frequente

In [ ]:
c = 0
c1 = 0

for i in range(kk):
  if resposta[i] == y_test[i]:
    c += 1

for i in range(kk1):
  if resposta1[i] == y1_test[i]:
    c1 += 1

print(f'Eficiência do algoritmo sem a retirada: {round((c/len(d))*100,2)}% \n Eficiência do algoritmo com a retirada: {round((c1/len(d1))*100,2)}%')

Eficiência do algoritmo sem a retirada: 39.25% 
 Eficiência do algoritmo com a retirada: 47.11%


Foi visto que a retirada das categorias com poucas observações não alterou muito na eficiência do algoritmo, ainda que proponha melhoria. O que mudou nesses últimos resultados, porque outros testes apresentaram resultados com eficiências mais distintaas, exemplo:

Eficiência do algoritmo sem a retirada: 27.1% 
Eficiência do algoritmo com a retirada: 45.6%